In [1]:
import json
import torch
import jsonlines
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

/root/miniconda3/envs/FT/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "MiniCPM-2B-sft-fp32"
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(
    path, torch_dtype=torch.bfloat16, device_map="cuda", trust_remote_code=True
)

In [3]:
res, history = model.chat(tokenizer, query="<user>Choose the correct option according to the following text | Question: Why is HGH and steroid use rampant in track ? | Options: Option_0:Because we have to assume that someone is using steroids because of an outstanding feat ., Option_1:None of the above choices ., Option_2:Because any incredible individual achievement is questioned immediately ., Option_3:Because it 's an American professional sport.\n Following is the context.\n HGH and steroid use is rampant in track , and , just like in most American professional sports , any incredible individual achievement is questioned immediately , which is a sad state of affairs . The fact that we , as a human population have to assume that someone is using steroids because of an outstanding feat is horrible . What about the people who actually train their hearts out to achieve the same success that the users are trying to achieve ? It is n't fair to the men and women that dedicate their lives to achieving glory the right way , hard work and determination.\n<assistant>", max_length=1024, top_p=0.5)
res, history

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


('Option_2: Because any incredible individual achievement is questioned immediately.',
 [{'role': 'user',
   'content': "<user>Choose the correct option according to the following text | Question: Why is HGH and steroid use rampant in track ? | Options: Option_0:Because we have to assume that someone is using steroids because of an outstanding feat ., Option_1:None of the above choices ., Option_2:Because any incredible individual achievement is questioned immediately ., Option_3:Because it 's an American professional sport.\n Following is the context.\n HGH and steroid use is rampant in track , and , just like in most American professional sports , any incredible individual achievement is questioned immediately , which is a sad state of affairs . The fact that we , as a human population have to assume that someone is using steroids because of an outstanding feat is horrible . What about the people who actually train their hearts out to achieve the same success that the users are tryin

In [4]:
test_sample_list = []

# 使用 jsonlines 库打开文件并逐行读取
with jsonlines.open("test_data/Cosmos_test.jsonl", 'r') as reader:
    for obj in reader:
        test_sample_list.append(obj)

In [5]:
output_list = []
for sample in tqdm(test_sample_list):
    res, history = model.chat(tokenizer, query="<user>{}<assistant>".format(sample["messages"][0]["content"]), max_length=1500, top_p=0.5, temperature=0.8)
    output_list.append(res)

 94%|█████████▍| 6552/6963 [40:47<02:15,  3.03it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|██████████| 6963/6963 [43:06<00:00,  2.69it/s]


In [7]:
output_file = 'output_cos_baseline.json'

# 将 output_list 写入 JSON 文件
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_list, f, ensure_ascii=False, indent=4)